#Home Work 4 

References:


1.   https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#build
2.  Custom Gradient and Layers: https://stackoverflow.com/questions/56657993/how-to-create-a-keras-layer-with-a-custom-gradient-in-tf2-0

3. https://goodboychan.github.io/python/coursera/tensorflow/deeplearning.ai/2022/02/08/02-Tensorflow2-Custom-Layers.html#Activation-in-a-custom-layer

Error Solving Ref:

1. https://stackoverflow.com/questions/56657993/how-to-create-a-keras-layer-with-a-custom-gradient-in-tf2-0

2. https://stackoverflow.com/questions/49029105/get-weights-from-keras-layer

3. https://stackoverflow.com/questions/67085779/typeerror-fit-got-an-unexpected-keyword-argument-test
4. https://stackoverflow.com/questions/57892492/why-the-gradient-of-categorical-crossentropy-loss-with-respect-to-logits-is-0-wi

5. Loss had to be spare categorical cross entropy. Else gives error with shapes.

Discussed and collaborated with Ramki and Sushant


In [2]:

import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Activation, Flatten
from librosa.core import stft,istft
import soundfile as sf
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

#Load Data

In [3]:
data = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = data.load_data()

print('dimensions of X_train : {}'.format(X_train.shape))
print('dimensions of y_train : {}'.format(y_train.shape))
print('dimensions of X_test  : {}'.format(X_test.shape))
print('dimensions of y_test  : {}'.format(y_test.shape))

11490434/11490434 [==============================] - 0s 0us/step
dimensions of X_train : (60000, 28, 28)
dimensions of y_train : (60000,)
dimensions of X_test  : (10000, 28, 28)
dimensions of y_test  : (10000,)


##Split Data

In [5]:
from sklearn.model_selection import train_test_split

X_train2,X_val,y_train2,y_val = train_test_split(X_train,y_train,train_size = 0.8,random_state= None, shuffle=False, stratify=None)

print('dimensions of X_train2 : {}'.format(X_train2.shape))
print('dimensions of y_train2 : {}'.format(y_train2.shape))
print('dimensions of X_val  : {}'.format(X_val.shape))
print('dimensions of y_val  : {}'.format(y_val.shape))

dimensions of X_train2 : (48000, 28, 28)
dimensions of y_train2 : (48000,)
dimensions of X_val  : (12000, 28, 28)
dimensions of y_val  : (12000,)


##Load Baseline Model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
baselinemodel = tf.keras.Sequential([
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28,28)),
    tf.keras.layers.Dense(units=1024,activation='relu', name="target_layer1"),
    tf.keras.layers.Dense(units=1024,activation='relu', name="target_layer2"),
    tf.keras.layers.Dense(units=1024,activation='relu', name="target_layer3"),
    tf.keras.layers.Dense(units=1024,activation='relu', name="target_layer4"),
    tf.keras.layers.Dense(units=1024,activation='relu', name="target_layer5"),
    tf.keras.layers.Dense(units=10,activation='softmax')
  ])

In [10]:
baselinemodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 784)               0         
                                                                 
 target_layer1 (Dense)       (None, 1024)              803840    
                                                                 
 target_layer2 (Dense)       (None, 1024)              1049600   
                                                                 
 target_layer3 (Dense)       (None, 1024)              1049600   
                                                                 
 target_layer4 (Dense)       (None, 1024)              1049600   
                                                                 
 target_layer5 (Dense)       (None, 1024)              1049600   
                                                                 
 dense (Dense)               (None, 10)                1

In [11]:
baselinemodel.compile(optimizer = 'adam',
              loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),metrics='accuracy')

In [12]:
history1 = baselinemodel.fit(X_train2,y_train2,epochs = 30, batch_size = 256,validation_data = (X_val,y_val))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


188/188 [==============================] - 4s 4ms/step - loss: 4.0372 - accuracy: 0.8587 - val_loss: 0.1594 - val_accuracy: 0.9515
Epoch 2/30
188/188 [==============================] - 1s 3ms/step - loss: 0.1221 - accuracy: 0.9623 - val_loss: 0.1256 - val_accuracy: 0.9637
Epoch 3/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0800 - accuracy: 0.9755 - val_loss: 0.1635 - val_accuracy: 0.9503
Epoch 4/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0595 - accuracy: 0.9814 - val_loss: 0.1270 - val_accuracy: 0.9649
Epoch 5/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0512 - accuracy: 0.9846 - val_loss: 0.1227 - val_accuracy: 0.9657
Epoch 6/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0434 - accuracy: 0.9862 - val_loss: 0.1295 - val_accuracy: 0.9684
Epoch 7/30
188/188 [==============================] - 1s 4ms/step - loss: 0.0446 - accuracy: 0.9861 - val_loss: 0.1183 - val_accuracy: 0.9677
Epoch 8/30
188/18

In [13]:
accuracy = model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1432 - accuracy: 0.9747


In [14]:
baselinemodel.layers

In [15]:
baselinemodel.get_weights()[0][1]

array([-0.04912192, -0.02060894,  0.00868594, ...,  0.05099872,
        0.03124828, -0.0214364 ], dtype=float32)

In [16]:
W1 = baselinemodel.get_layer(name = "target_layer1").get_weights()[0]
W2 = baselinemodel.get_layer(name = "target_layer2").get_weights()[0]
W3 = baselinemodel.get_layer(name = "target_layer3").get_weights()[0]
W4 = baselinemodel.get_layer(name = "target_layer4").get_weights()[0]
W5 = baselinemodel.get_layer(name = "target_layer5").get_weights()[0]



In [17]:
W5.shape

(1024, 1024)

In [31]:
print(W1.shape)
print(W2.shape)
print(W5.shape)

(784, 1024)
(1024, 1024)
(1024, 1024)


In [18]:
B1 = baselinemodel.get_layer(name = "target_layer1").get_weights()[1]
B2 = baselinemodel.get_layer(name = "target_layer2").get_weights()[1]
B3 = baselinemodel.get_layer(name = "target_layer3").get_weights()[1]
B4 = baselinemodel.get_layer(name = "target_layer4").get_weights()[1]
B5 = baselinemodel.get_layer(name = "target_layer5").get_weights()[1]

In [32]:
print(B1.shape)
print(B2.shape)
print(B5.shape)

(1024,)
(1024,)
(1024,)


Note: Shapes of weights are changing. We cannot hard code 784. We need to use input_shape[-1] as given in documentation.


But its giving error 

#Custom Definitions

In [19]:
# @tf.custom_gradient
# def customgrad(weight):
#   sum, direction, value = tf.linalg.svd(weight)
#   S = tf.linalg.diag(sum[0:20,0:20])
#   D = direction[:,0:20] 
#   V = tf.transpose(value[:,0:20])

#   W_hat = tf.matmul(tf.matmul(D, S), V)
  
#   #backprop
#   def grad(dy):
#     gradient = dy
#     return gradient

#   return W_hat, grad









# class CustomDense(tf.keras.layers.Layer):
#   def __init__(self, units):
#       super(CustomDense, self).__init__()
#       self.units = units
#       self.activation = tf.nn.relu
#       #self.weight = tf.Variable(W)


#   def build(self, input_shape):
#     self.w = self.add_weight("weight",shape=[int(input_shape[-1]), self.units])
#     self.bias = self.add_weight("bias", shape = [self.units,])
#     #self.w = tf.Variable(self.W)

#       #self.bias = self.add_weight(B)

#   def call(self, inputs):
#     W_hat = customgrad(self.w)
#     return self.activation((tf.matmul(inputs, W_hat) + self.bias))






  

Defined Weights in _init_ layer as  build layer will addweights and bias only based on input shape.

I could not use set_weight(baselinemodel.get_weights()) and set the weights and use input shape as for some reason while compiling the model i faced an shape error.

Hence i am manually setting the weights by fetching the weights and biases outside and passing it as a layer attribute.

I tried that in above code, Huge Issues !

In [30]:

class CustomDense(tf.keras.layers.Layer):
  def __init__(self, units, activation,W,B):
        super(CustomDense, self).__init__()
        self.units = units
        W = tf.Variable(W, name='Weights')
        B = tf.Variable(B, name='Biases')
        self.w = W
        self.b = B
        self.activation=activation


  # def weight_biases(self, input_shape,W,B):
  #       #w_init = tf.random_normal_initializer()


  def call(self, inputs):
        W_svd = customgrad(self.w)
        
        return self.activation(tf.matmul(inputs,W_svd) + self.b)



@tf.custom_gradient
def customgrad(weight):
  sum , direction, value = tf.linalg.svd(weight)
  S = tf.linalg.diag(sum)

  V = tf.transpose(value)

  W = tf.matmul(tf.matmul(direction[:,:20], S[:20,:20]), V[:20,:])

    #backprop
  def grad(dy):
    gradient = dy
    return gradient

  return W, grad

In [21]:
activation = tf.nn.relu

In [22]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28,28)),
    CustomDense(1024,activation,W1,B1),
    CustomDense(1024,activation,W2,B2),
    CustomDense(1024,activation,W3,B3),
    CustomDense(1024,activation,W4,B4),
    CustomDense(1024,activation,W5,B5),
    
    tf.keras.layers.Dense(units=10,activation='softmax')
  ])

In [23]:
model3.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])



In [24]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 784)               0         
                                                                 
 custom_dense (CustomDense)  (None, 1024)              803840    
                                                                 
 custom_dense_1 (CustomDense  (None, 1024)             1049600   
 )                                                               
                                                                 
 custom_dense_2 (CustomDense  (None, 1024)             1049600   
 )                                                               
                                                                 
 custom_dense_3 (CustomDense  (None, 1024)             1049600   
 )                                                               
                                                      

In [25]:
X_train2.shape

(48000, 28, 28)

In [26]:
y_train2.shape

(48000,)

In [27]:
history = model3.fit(X_train2, y_train2, batch_size=256, epochs=10,verbose=True)

Epoch 1/10
188/188 [==============================] - 378s 2s/step - loss: 0.2453 - accuracy: 0.9299
Epoch 2/10
188/188 [==============================] - 375s 2s/step - loss: 0.1200 - accuracy: 0.9655
Epoch 3/10
188/188 [==============================] - 375s 2s/step - loss: 0.0979 - accuracy: 0.9721
Epoch 4/10
188/188 [==============================] - 375s 2s/step - loss: 0.0883 - accuracy: 0.9737
Epoch 5/10
188/188 [==============================] - 374s 2s/step - loss: 0.0883 - accuracy: 0.9744
Epoch 6/10
188/188 [==============================] - 373s 2s/step - loss: 0.0833 - accuracy: 0.9760
Epoch 7/10
188/188 [==============================] - 372s 2s/step - loss: 0.0784 - accuracy: 0.9768
Epoch 8/10
188/188 [==============================] - 373s 2s/step - loss: 0.0792 - accuracy: 0.9770
Epoch 9/10
188/188 [==============================] - 371s 2s/step - loss: 0.0724 - accuracy: 0.9786
Epoch 10/10
188/188 [==============================] - 371s 2s/step - loss: 0.0684 - accura

In [29]:
accuracy  = model3.evaluate(X_test, y_test,verbose=True,batch_size=256)

40/40 [==============================] - 79s 2s/step - loss: 0.1292 - accuracy: 0.9664


In [ ]:
%%shell
jupyter nbconvert --to html 